<a href="https://colab.research.google.com/github/xxFirefly/mpc_rl_capsubot/blob/new_feature/example_mpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of model usage

In [1]:
#%matplotlib
if 'google.colab' in str(get_ipython()):
  import os
  if (os.path.split(os.getcwd())[-1] != 'mpc_rl_capsubot'):
    import os
    !git clone https://github.com/xxFirefly/mpc_rl_capsubot  
    !pip install casadi==3.5.5
    os.chdir("mpc_rl_capsubot")
  !git pull

from envs.CapsubotEnv import CapsubotEnv
import numpy as np
from time import time
import matplotlib.pyplot as plt
import scipy.constants

# https://github.com/MMehrez/MPC-and-MHE-implementation-in-MATLAB-using-Casadi/blob/master/workshop_github/Python_Implementation/mpc_code.py

# pip install casadi
from casadi import *

try:
    env.close()
except Exception:
    pass

env = CapsubotEnv()

fatal: destination path 'mpc_rl_capsubot' already exists and is not an empty directory.
Already up to date.


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


# Helper functions

In [2]:
def shift_timestep(step_horizon, t0, state_init, u, f):
    f_value = f(state_init, u[:, 0])
    next_state = DM.full(state_init + (step_horizon * f_value))

    t0 = t0 + step_horizon
    u0 = horzcat(
        u[:, 1:],
        reshape(u[:, -1], -1, 1)
    )

    return t0, next_state, u0


def DM2Arr(dm):
    return np.array(dm.full())

# State and control variables.

In [3]:
# Based on the exapmle 
# https://github.com/MMehrez/MPC-and-MHE-implementation-in-MATLAB-using-Casadi/blob/master/workshop_github/Python_Implementation/mpc_code.py

x = SX.sym("x")
xi = SX.sym("xi")
x_dot = SX.sym("x_dot")
xi_dot = SX.sym("xi_dot")
states = vertcat(x, x_dot, xi, xi_dot)
n_states = states.numel()
force = SX.sym("F")
controls = vertcat(force)
n_controls = controls.numel()


# MPC and optimal control parameters.

---



In [4]:
# Setting matrix_weights' variables. For objective function??
Q_x = 1
Q_x_dot = 1
Q_xi = 1
Q_xi_dot = 1
R_1 = 1

step_horizon = 0.01 # Time between steps in seconds.
N = 10 #int(0.1 / env.dt)  # Prediction horizon.
sim_time = 100

F_max = 1.25 # Need to add proper force model
F_min = 0 

# Initial and target values.
x_init = 0
x_dot_init = 0
xi_init = 0
xi_dot_init = 0
x_target = 0.2
x_dot_target = 0
xi_target = 0
xi_dot_target = 0



# A vector that represents the states over the optimization problem.
X = SX.sym("X", n_states, (N + 1))
# Decision variables (controls)
U = SX.sym("U", n_controls, N)
# Parameters (which include the initial state and the reference state)
P = SX.sym("P", n_states + n_states)
# state weights matrix (Q_X, Q_Y, Q_THETA)
Q = diagcat(Q_x, Q_x_dot, Q_xi, Q_xi_dot)
# controls weights matrix
R = diagcat(R_1)

# Model

In [5]:
friction = env.friction_model(x_dot)
x_acc = (env.stiffness * xi - force + friction) / env.M
xi_acc = (-env.stiffness * xi + force) / env.m - x_acc

rhs = vertcat(x_dot, x_acc, xi_dot, xi_acc)  # System r.h.s

# Nonlinear mapping function f(x,u).
f = Function("f", [states, controls], [rhs])

In [6]:
cost_fn = 0 # Cost function.
g = X[:, 0] - P[:n_states] # Constraints in the equation. 

In [7]:
# Runge kutta.
for k in range(N):
    st = X[:, k]
    con = U[:, k]
    cost_fn = cost_fn \
        + (st - P[n_states:]).T @ Q @ (st - P[n_states:]) \
        + con.T @ R @ con
    st_next = X[:, k+1]
    k1 = f(st, con)
    k2 = f(st + step_horizon/2*k1, con)
    k3 = f(st + step_horizon/2*k2, con)
    k4 = f(st + step_horizon * k3, con)
    st_next_RK4 = st + (step_horizon / 6) * (k1 + 2 * k2 + 2 * k3 + k4)
    g = vertcat(g, st_next - st_next_RK4)

# Problem solving

In [8]:
OPT_variables = vertcat(
    X.reshape((-1, 1)), U.reshape((-1, 1))  # Example: 3x11 ---> 33x1 where 3=states, 11=N+1
)
nlp_prob = {"f": cost_fn, "x": OPT_variables, "g": g, "p": P}

opts = {
    "ipopt": {
        "max_iter": 2000,
        "print_level": 0,
        "acceptable_tol": 1e-8,
        "acceptable_obj_change_tol": 1e-6,
    },
    "print_time": 0,
}

solver = nlpsol("solver", "ipopt", nlp_prob, opts)

lbx = DM.zeros((n_states * (N + 1) + n_controls * N, 1))
ubx = DM.zeros((n_states * (N + 1) + n_controls * N, 1))

lbx[0 : n_states * (N + 1) : n_states] = -inf  # X lower bound
lbx[1 : n_states * (N + 1) : n_states] = -inf  # x_dot lower bound
lbx[2 : n_states * (N + 1) : n_states] = -inf  # xi lower bound
lbx[3 : n_states * (N + 1) : n_states] = -inf  # xi_dot lower bound


ubx[0 : n_states * (N + 1) : n_states] = inf  # X lower bound
ubx[1 : n_states * (N + 1) : n_states] = inf  # x_dot upper bound
ubx[2 : n_states * (N + 1) : n_states] = inf  # xi lower bound
ubx[3 : n_states * (N + 1) : n_states] = inf  # xi_dot upper bound


lbx[n_states * (N + 1) :] = F_min  # v lower bound for F.
ubx[n_states * (N + 1) :] = F_max  # v upper bound for F


args = {
    "lbg": DM.zeros((n_states * (N + 1), 1)),  # constraints lower bound
    "ubg": DM.zeros((n_states * (N + 1), 1)),  # constraints upper bound
    "lbx": lbx,
    "ubx": ubx,
}

In [9]:
t0 = 0
state_init = DM([x_init, x_dot_init, xi_init, xi_dot_init])  # initial state
state_target = DM([x_target, x_dot_target, xi_target, xi_dot_target])  # target state

t = DM(t0)

u0 = DM.zeros((n_controls, N))  # initial control
X0 = repmat(state_init, 1, N + 1)  # initial state full


mpc_iter = 0
cat_states = DM2Arr(X0)
cat_controls = DM2Arr(u0[:, 0])
times = np.array([[0]])

In [ ]:
main_loop = time()  # return time in sec
while (norm_2(state_init - state_target) > 1e-1) and (mpc_iter * step_horizon < sim_time):
    t1 = time()
    args["p"] = vertcat(state_init, state_target)  # current state  # target state
    # optimization variable current state
    args["x0"] = vertcat(reshape(X0, n_states * (N + 1), 1), reshape(u0, n_controls * N, 1))


    sol = solver(
        x0=args["x0"],
        lbx=args["lbx"],
        ubx=args["ubx"],
        lbg=args["lbg"],
        ubg=args["ubg"],
        p=args["p"],
    )

    u = reshape(sol["x"][n_states * (N + 1) :], n_controls, N)
    X0 = reshape(sol["x"][: n_states * (N + 1)], n_states, N + 1)

    cat_states = np.dstack((cat_states, DM2Arr(X0)))
    # print(cat_state)

    cat_controls = np.vstack((cat_controls, DM2Arr(u[:, 0])))
    t = np.vstack((t, t0))

    t0, state_init, u0 = shift_timestep(step_horizon, t0, state_init, u, f)

    X0 = horzcat(X0[:, 1:], reshape(X0[:, -1], -1, 1))
    print(X0)

    # xx ...
    t2 = time()
    # if (mpc_iter * step_horizon) % 0.005 <= step_horizon:
    #     print("iter n:", mpc_iter)
    #     print("iter time:", t2 - t1)
    #     print("simulated time:", mpc_iter * step_horizon)
    #     print("pose error:", norm_2(state_init - state_target))
    #     print(state_init)
    # print("iter n:", mpc_iter)
    # print("iter time:", t2 - t1)
    # print("simulated time:", mpc_iter * step_horizon)
    # print("pose error:", norm_2(state_init - state_target))
    # print(state_init)

    times = np.vstack((times, t2 - t1))

    mpc_iter = mpc_iter + 1

main_loop_time = time()
ss_error = ca.norm_2(state_init - state_target)

print("\n\n")
print("Total time: ", main_loop_time - main_loop)
print("avg iteration time: ", np.array(times).mean() * 1000, "ms")
print("final error: ", ss_error)




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************


[[0.199755, 0.199999, 0.199999, 0.2, 0.199999, 0.199999, 0.199999, 0.2, 0.199999, 0.199902, 0.199902], 
 [-4.92299e-09, -4.92056e-09, -4.96164e-09, 2.17953e-05, 2.24306e-05, 4.90928e-09, -4.87712e-09, 4.87011e-09, 8.40123e-07, 0.0117209, 0.0117209], 
 [-3.17814e-05, -3.17656e-05, -3.20331e-05, -1.43208e-05, -1.3953e-05, 3.18585e-05, -3.14829e-05, 3.16036e-05, 1.48345e-07, 8.86801e-05, 8.86801e-05], 
 [-1.47191e-06, -1.64595e-07, -1.44961e-07, -8.98459e-07, -2.43609e-08, -8.64869e-07, 1.75751e-06, -1.81383e-06, 9.18201e-07, -0.00954818, -0.00954818]]

[[0.000160438, 2.42891e-05, -0.000125957, -0.000264914

In [ ]:
states = np.array(states)

fig, axs = plt.subplots(2, 2, sharex=True)  # , sharex=True, sharey=True)
fig.suptitle("average speed: {:.3}".format(states[-1, 0] / ts[-1]))
axs[0, 0].plot(ts, states[:, 0], label="x")
axs[0, 0].set_title("x")
axs[1, 0].plot(ts, states[:, 1], label="dx")
axs[1, 0].set_title("dx")
axs[0, 1].plot(ts, states[:, 2] + states[:, 0], label="xi")
axs[1, 0].set_title("xi")
axs[1, 1].plot(ts, states[:, 3], label="dxi")
axs[1, 1].set_title("dxi")
t_shift = 3
axs[0, 0].set_xlim(t_shift, t_shift + 1)


plt.figure()
plt.plot(ts, actions, label="F")
plt.xlim(t_shift, t_shift + 1)
plt.legend()
plt.show()

